In [1]:
import numpy as np 
from scipy.integrate import trapz as tp 
from funcs.aflare1 import aflare1

In [2]:
import pandas as pd
#import data frames with the flare parameters of davenport model (ampl, fwhm, tpeak)
#fitted by emcee approach McKey 2013 
df_g = pd.read_pickle("/home/aaronmaas/Flare_Notebooks/flare_tables/trappist_1/g_filter/flareprofiles_g.pkl")
df_r = pd.read_pickle("/home/aaronmaas/Flare_Notebooks/flare_tables/trappist_1/r_filter/flareprofiles_r.pkl")
df_i = pd.read_pickle("/home/aaronmaas/Flare_Notebooks/flare_tables/trappist_1/i_filter/flareprofiles_i.pkl")
df_z = pd.read_pickle("/home/aaronmaas/Flare_Notebooks/flare_tables/trappist_1/z_filter/flareprofiles_z.pkl")


#dataframe for the different flares and observation time
dfs = [df_g,df_r,df_i,df_z]
timeobs = [np.linspace(df_g["timestart"][0],df_g["timestop"][0],1000),np.linspace(df_g["timestart"][1],df_g["timestop"][1],1000)]

In [3]:
from funcs._brightness_luminosity import brightnessnorm_flare

In [4]:
brightnessnorm_flare(timeobs,dfs)

NameError: name 'brightness' is not defined

In [8]:
brightness, brightnesserror = [],[]

#calculation of brightness for all flare events in the dataframe
#above shown calculation implemented
i = 0
for df in dfs:
    for flare in range(len(timeobs)):
        #time where observation is different then zero
        timetime = timeobs[flare][aflare1(timeobs[flare], df.Tpeak[flare], df.Fwhm[flare], df.Ampl[flare]) > 0]
        T = timetime[-1] - timetime[0] 
        #brightness of flare in bandpass devided by total flare time, to get rid of time unit
        brightness.append(tp(aflare1(timeobs[flare], df.Tpeak[flare], df.Fwhm[flare], df.Ampl[flare]))  * 1/T  )
        brightnesserror.append(tp(aflare1(timeobs[flare], df.Tpeak[flare], np.max([df.sFwhm[flare],df.sfwhm[flare]]), np.max([df.sAmpl[flare],df.sampl[flare]]))) * 1/T)
    i = i + 1 

#every second entry belongs to the same flare therefore slice [0::2]
brightness = np.asarray(brightness)
brightnesserror = np.asarray(brightnesserror)

In [9]:
brightness

array([4.52506984e+02, 1.42712541e+01, 1.51050116e+02, 6.05080978e+00,
       1.01800624e+01, 4.24348275e-01, 2.82384713e+00, 5.77041610e-02])

In [30]:
with open("../results/values/brightness_flare1.txt", "w+") as f:
    f.write(str(brightness[0::2]))
    f.write(str(brightnesserror[0::2]))

In [31]:
with open("../results/values/brightness_flare2.txt", "w+") as f:
    f.write(str(brightness[1::2]))
    f.write(str(brightnesserror[1::2]))